In [1]:
import sys
sys.path.append("..")
from plotters import plot_fit, plot_sensitivity_analysis
from plotters.plot_fit import plot_fit_multiple, plot_runs
from plotters.plot_sensitivity import generate_ethnicity_df, generate_social_df, generate_age_df, generate_district_df, get_newton_step, plot_sensitivity_analysis_multiple
from plotters.plot_sensitivity import get_sensitivity_ethnicity_multiple, get_sensitivity_age_multiple, get_sensitivity_social_multiple, get_sensitivity_district_multiple
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import yaml
import datetime
from pathlib import Path
from matplotlib import dates as mdates
from collections import defaultdict
import matplotlib as mpl
from tqdm import tqdm
import torch
import geopandas as gpd
from matplotlib.colors import LogNorm, SymLogNorm
mpl.rcParams["figure.dpi"] = 100

from grad_june import Runner

/var/folders/s5/jmvhvqns52q3ysypfjykg6y40000gr/T/ipykernel_69306/2827350927.py:18: UserWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas will still use PyGEOS by default for now. To force to use and test Shapely 2.0, you have to set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In a future release, GeoPandas will switch to using Shapely by default. If you are using PyGEOS directly (calling PyGEOS functions on geometries from GeoPandas), this will then stop working and you are encouraged to migrate from PyGEOS to Shapely 2.0 (https://shapely.readthedocs.io/en/latest/migration_pygeos.html).
  import geopandas as gpd


In [2]:
fit_files = np.array(list(Path("../Data/June/london_fits").glob("*.yaml")))[[2, 3, 4, 6, 11]]
fit_files

array([PosixPath('../Data/June/london_fits/london_fit_011.yaml'),
       PosixPath('../Data/June/london_fits/london_fit_007.yaml'),
       PosixPath('../Data/June/london_fits/london_fit_000.yaml'),
       PosixPath('../Data/June/london_fits/london_fit_017.yaml'),
       PosixPath('../Data/June/london_fits/london_fit_019.yaml')],
      dtype=object)

In [3]:
plot_runs(fit_files)

  0%|          | 0/5 [00:00<?, ?it/s]


AssertionError: Torch not compiled with CUDA enabled

In [ ]:
plot_fit_multiple(fit_files, window=7, days=61, errors=True)

In [ ]:
plot_sensitivity_analysis_multiple(fit_files, days=5)

# Ethnicity plots

In [ ]:
#df_ethn = generate_ethnicity_df(r, n=1)
df_ethn = get_sensitivity_ethnicity_multiple(fit_files, 3)

In [ ]:
toplot = pd.concat(df_ethn)
toplot = toplot.loc[["household", "school", "company", "university"]]
toplot = toplot.rename(columns={"A" : "White", "B" : "Mixed", "C": "Asian", "D": "Black", "E": "Other"})
toplot = toplot.rename({"company": "Company", 'school' : 'School', "household" : "Household", "university" : "University"})
mean = toplot.groupby(toplot.index).mean()
std = toplot.groupby(toplot.index).std()
fig, ax = plt.subplots()
mean.plot.barh(ax=ax, xerr=std, capsize=3, width=0.8)
ax.set_xlabel(r"Sensitivity of $f^d$")
ax.legend(title="Ethnicity")
fig.savefig("../figures/sensitivity_ethnicity.png", dpi=150, bbox_inches="tight")


In [ ]:
#df_social = generate_social_df(r, n=1)
df_social = get_sensitivity_social_multiple(fit_files, 3)

In [ ]:
toplot = pd.concat(df_social)
toplot = toplot.loc[["household", "school", "company", "university"]]
toplot = toplot.rename({"company": "Company", 'school' : 'School', "household" : "Household", "university" : "University"})
toplot = toplot.rename(columns={1: "1 (least deprived)", 5: "5 (most deprived)"})

mean = toplot.groupby(toplot.index).mean()
std = toplot.groupby(toplot.index).std()
fig, ax = plt.subplots()
mean.plot.barh(ax=ax, xerr=std, capsize=3, width=0.8)
ax.set_xlabel(r"Sensitivity of $f^d$")
ax.legend(title="IMD quintile")
fig.savefig("../figures/sensitivity_social.png", dpi=150, bbox_inches="tight")

In [ ]:
#df_age = generate_age_df(r, date="2020-03-03", n=1)
df_age = get_sensitivity_age_multiple(fit_files, 3)

In [ ]:
toplot = pd.concat(df_age)
toplot = toplot.loc[["household", "school", "company", "university"]]
toplot = toplot.rename({"company": "Company", 'school' : 'School', "household" : "Household", "university": "University"})
toplot = toplot.rename(columns={18: "0-17", 25 : "18-24", 35 : "25-34", 45: "35-44", 55 : "45-54", 65: "55-64", 75 : "65-74", 100: "75+"})

fig, ax = plt.subplots()
mean = toplot.groupby(toplot.index).mean()
std = toplot.groupby(toplot.index).std()
mean.plot.barh(ax=ax, xerr=std, capsize=3, width=0.8)
ax.set_xlabel(r"Sensitivity of $f^d$")
ax.legend(title="Age bin", ncol=2)
fig.savefig("../figures/sensitivity_age.png", dpi=150, bbox_inches="tight")

In [ ]:
district_ids = [276, 277, 278, 279, 280, 281, 282, 283, 284, 285, 286, 287, 288, 289,
        290, 291, 292, 293, 294, 295, 296, 297, 298, 299, 300, 301, 302, 303,
        304, 305, 306, 307, 308]
district_df = pd.read_csv("../../GradABM/Data/June/area_district.csv")
district_df = district_df.loc[district_df.id.isin(district_ids)]
district_df.set_index("id", inplace=True)
district_df.sort_index(inplace=True)
district_names = district_df["ladnm"].drop_duplicates().values

In [ ]:
shapefiles = gpd.read_file("../statistical-gis-boundaries-london/ESRI/London_Borough_Excluding_MHW.shp")
shapefiles = shapefiles.set_index("NAME")

In [ ]:
#df_district = generate_district_df(r, n=5)
df_district = get_sensitivity_district_multiple(fit_files, 3)

In [ ]:
df_district = pd.concat(df_district)
df_district_std = df_district.groupby(df_district.index).std()
df_district = df_district.groupby(df_district.index).mean()

df_district_std.columns = district_names
df_district.columns = district_names
df_district = df_district.transpose()
df_district_std = df_district_std.transpose()

In [ ]:
geo_df = pd.merge(shapefiles, df_district, left_index=True, right_index=True)
geo_df_std = pd.merge(shapefiles, df_district_std, left_index=True, right_index=True)

In [ ]:
f, ax = plt.subplots(2,2)

geo_df.plot("household", ax=ax[0,0], norm=SymLogNorm(1e-4))#, legend=True, norm=SymLogNorm(1e-4, vmin=-1e-1, vmax=1e-1))
geo_df.plot("school", ax=ax[0,1], norm=SymLogNorm(1e-4))
geo_df.plot("company", ax=ax[1,0], norm=SymLogNorm(1e-4))
geo_df.plot("university", ax=ax[1,1], norm=SymLogNorm(1e-4))


for i in range(2):
    for j in range(2):
        ax[i, j].set_xticks([])
        ax[i, j].set_yticks([])
    
#ax[1].set_title("Sensitivity by district")
ax[0,0].set_title("Household")
ax[0,1].set_title("School")
ax[1,0].set_title("Company")
ax[1,1].set_title("University")

plt.subplots_adjust(wspace=0.05, hspace=0.2)
f.savefig("../figures/sensitivity_district.png", dpi=150, bbox_inches="tight")

# Optimal policy setup

In [ ]:
def get_gradient(file, date_str):
    date = datetime.datetime.strptime(date_str, "%Y-%m-%d")
    n_days = (date - datetime.datetime.strptime("2020-03-01", "%Y-%m-%d")).days
    params = yaml.safe_load(open(file))
    params["timer"]["total_days"] = n_days
    params["system"]["device"] = "cpu"
    r = Runner.from_parameters(params)

    for network in r.model.infection_networks.networks:
        r.model.infection_networks.networks[network].log_beta = torch.nn.Parameter(r.model.infection_networks.networks[network].log_beta)
    results, _ = r()
    cases = results["cases_per_timestep"][-1]
    cases.backward()
    ret = {}
    norm = 0
    for network in r.model.infection_networks.networks:
        v = r.model.infection_networks.networks[network].log_beta.grad.item()
        norm += v**2
        ret[network] = v
    norm = np.sqrt(norm)
    for key in ret:
        ret[key] = ret[key] / norm
    return ret



In [ ]:
def generate_params(file, gradients):
    params = yaml.safe_load(open(file))
    params["timer"]["total_days"] = 60
    params["system"]["device"] = "cuda:8"
    params["policies"]["interaction"]["social_distancing"] = {}
    params["policies"]["interaction"]["social_distancing"][1] = {}
    params["policies"]["interaction"]["social_distancing"][1]["start_date"] = "2020-03-16"
    params["policies"]["interaction"]["social_distancing"][1]["end_date"] = "2021-03-16"
    params["policies"]["interaction"]["social_distancing"][1]["beta_factors"] = {}
    for key in gradients:
        if key == "household":
            continue
        old_beta = 10**(params["networks"][key]["log_beta"])
        delta_beta = gradients[key] * old_beta
        new_beta = old_beta -  delta_beta # derivative of the log
        factor = new_beta / old_beta
        params["policies"]["interaction"]["social_distancing"][1]["beta_factors"][key] = factor
    return params

def run_for_policy(params):
    with torch.no_grad():
        r = Runner.from_parameters(params)
        results, _ = r()
        return np.array(results["dates"]), results["cases_per_timestep"].cpu().numpy()
    
def run_vanilla(file):
    params = yaml.safe_load(open(file))
    params["timer"]["total_days"] = 60
    params["system"]["device"] = "cuda:8"
    with torch.no_grad():
        r = Runner.from_parameters(params)
        results, _ = r()
        return np.array(results["dates"]), results["cases_per_timestep"].cpu().numpy()
    
def run_for_files(files):
    results_opt = []
    results_naive = []
    results_vanilla = []
    results_nop = []
    for file in tqdm(files):
        gradient = get_gradient(file, "2020-03-15")
        naive_grad = {key: 1 / len(gradient) for key in gradient}
        
        params_opt = generate_params(file, gradient)
        dates_opt, res_opt = run_for_policy(params_opt)
        results_opt.append(res_opt)
        
        params_naive = generate_params(file, naive_grad)
        dates_naive, res_naive = run_for_policy(params_naive)
        results_naive.append(res_naive)
        
        dates_vanilla, res_vanilla = run_vanilla(file)
        results_vanilla.append(res_vanilla)
        
        params_nop = generate_params(file, {key: 0.0 for key in gradient})
        dates_nop, res_nop = run_for_policy(params_nop)
        results_nop.append(res_nop)
        
    return dates_opt, np.array(results_opt), np.array(results_naive), np.array(results_vanilla), np.array(results_nop)

In [ ]:
dates, results_opt, results_naive, results_vanilla, results_nop = run_for_files(fit_files)

In [ ]:
results_opt = np.array(results_opt)
results_naive = np.array(results_naive)
results_vanilla = np.array(results_vanilla)
results_nop = np.array(results_nop)

In [ ]:
f, ax = plt.subplots()
ax.plot(dates, results_opt.mean(0), label = "Optimal cost-effective lockdown")
ax.plot(dates, results_naive.mean(0), label = "Naive cost-effective lockdown")
ax.plot(dates, results_vanilla.mean(0), label = "Real lockdown")
ax.plot(dates, results_nop.mean(0), label = "No lockdown")

alpha = 0.25
ax.fill_between(dates, results_opt.mean(0) - results_opt.std(0), results_opt.mean(0) + results_opt.std(0), alpha=alpha, linewidth=0)
ax.fill_between(dates, results_naive.mean(0) - results_naive.std(0), results_naive.mean(0) + results_naive.std(0), alpha=alpha, linewidth=0)
ax.fill_between(dates, results_vanilla.mean(0) - results_vanilla.std(0), results_vanilla.mean(0) + results_vanilla.std(0), alpha=alpha, linewidth=0)
ax.fill_between(dates, results_nop.mean(0) - results_nop.std(0), results_nop.mean(0) + results_nop.std(0), alpha=alpha, linewidth=0)

ax.set_yscale("log")
fmt_month = mdates.MonthLocator()
ax.xaxis.set_major_locator(fmt_month)
ax.xaxis.set_major_formatter(mdates.DateFormatter("%b"))
ax.axvline(datetime.datetime.strptime("2020-03-15", "%Y-%m-%d"), color = "black", linestyle="--", linewidth=1)
ax.set_ylabel("Cumulative cases")
ax.legend()
f.savefig("../figures/optimal_lockdown.png", dpi=150, bbox_inches="tight")